# Publishing your maps and charts in github pages
Github pages enables you to publish content in your repo that are in html format. This includes html files generated from folium maps, kepler maps, and interactive plotly charts.

## Enable github pages

- follow the steps in [this tutorial](https://guides.github.com/features/pages/) to enable github pages

## Get arrest data

In [15]:
# to import data from LA Data portal
from sodapy import Socrata
import pandas as pd
import geopandas as gpd

# connect to the data portal
client = Socrata("data.lacity.org", None)

results = client.get("amvf-fr72", 
                     limit=50000,
                     where = "arst_date between '2020-09-01T00:00:00' and '2020-10-30T00:00:00'",
                     order='arst_date desc')

# Convert to pandas DataFrame
arrests = pd.DataFrame.from_records(results)

In [16]:
arrests.shape

(4360, 25)

In [17]:
# drop the unmapped rows
arrests.drop(arrests[arrests.lon==0].index,inplace=True)

In [18]:
# convert lat/lon to floats
arrests.lon = arrests.lon.astype('float')
arrests.lat = arrests.lat.astype('float')

In [19]:
arrests = arrests[['arst_date','grp_description','sex_cd','descent_cd','lat','lon']]

In [20]:
arrests.head()

arst_date                 grp_description sex_cd descent_cd  \
0  2020-09-26T00:00:00.000              Narcotic Drug Laws      M          B   
1  2020-09-26T00:00:00.000              Narcotic Drug Laws      M          B   
2  2020-09-26T00:00:00.000  Miscellaneous Other Violations      M          W   
3  2020-09-26T00:00:00.000             Weapon (carry/poss)      M          H   
4  2020-09-26T00:00:00.000       Moving Traffic Violations      M          W   

       lat       lon  
0  34.0515 -118.2658  
1  33.9420 -118.4095  
2  34.0837 -118.3543  
3  34.1758 -118.4052  
4  34.0430 -118.2571

## Save interactive elements as html pages

For interactive visualizations, you can save them out as html files from within the Jupyter Notebook environment.

### Folium

In [ ]:
# import folium, a leaflet python library
import folium

# create a list of lat/lon pair for UCLA
ucla = [34.0721237,-118.4440685]
m = folium.Map(location=ucla, zoom_start=15)

# save as html file
m.save('ucla.html')

### Kepler

In [21]:
from keplergl import KeplerGl
map = KeplerGl(height=600, width=800, data={'arrests':arrests})
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'arrests':                     arst_date                 grp_description sex_cd  \
0     2020-0…

In [39]:
map.save_to_html(file_name='arrests_kepler.html',read_only=True)

Map saved to arrests_kepler.html!


### Plotly


In [22]:
import plotly.express as px

In [28]:
# ok, group by date and charge, and let's get a count for each
arrests_by_type=arrests.groupby(['arst_date','grp_description']).size().reset_index(name='count')

In [41]:
# make a stacked bar chart
fig = px.bar(arrests_by_type,
           x='arst_date',
           y='count',
           color='grp_description', # this creates the "stack",
           labels={'arst_date':'Arrest Date','count':'Number of arrests','grp_description':'Charge type'},
           title = 'September 2020 LAPD Arrests by Charge Type'
      )
fig.show()

In [43]:
fig.write_html("arrests_stacked_plotly.html")

In [37]:
# map it
fig = px.scatter_mapbox(arrests, 
                        lat="lat", 
                        lon="lon", 
                        color="descent_cd",
                        labels={'descent_cd':'Race of person arrested'}
                       )
fig.update_layout(mapbox_style="carto-positron")
b
fig.show()

In [40]:
fig.write_html("arrests_map_plotly.html")

## Add to github pages

1. Download the html file to your local computer
1. Upload the html file to your github repo
  - Make sure to note the folder where you upload to.
1. Make sure you have enabled github pages
1. The path to your html file will be:


https://***your_github_username***.github.io/***name_of_your_repo***/***folder_path***(if not in root)/***your_html_file.html***

For example, to access the files created in this notebook from my repo, the links would be:


- https://yohman.github.io/up206a/Weeks/Week10/arrests_kepler.html
- https://yohman.github.io/up206a/Weeks/Week10/arrests_stacked_plotly.html
- https://yohman.github.io/up206a/Weeks/Week10/arrests_map_plotly.html
